# Numerical Analysis Project
Team: 아프지마

Member

* 20181628 성창훈
* 20181635 신명철

written by 성창훈, 신명철

# Paperspace Gradient Notebook

AutoSklearn은 패키지 스펙상 CPU 성능이 시간에 상당한 영향을 미친다고 한다.

그래서 이번 노트는 Google Colab 대신 CPU와 RAM 자원이 넉넉한 Paperspace Gradient Notebook을 이용했다.

In [1]:
!nvidia-smi

Mon May 24 19:48:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro M4000        On   | 00000000:00:05.0 Off |                  N/A |
| 46%   24C    P8    11W / 120W |     61MiB /  8126MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Packages
대다수의 Web-Based notebook 서비스들(Google Colab, Kaggle, Paperspace...)은 docker를 이용해 일관된 환경을 제공한다.

환경의 업데이트는 패키지의 업데이트보다 보수적으로 이루어지므로 보통 버전이 낮다.

이때 일부 최신 패키지는 높은 버전의 pip 버전을 요하므로, 반드시 패키지 관리 도구를 최신 버전으로 업데이트하여 사용한다.

업데이트 후에는 kernel을 다시 시작한다. (초기화가 아님에 유의한다)

In [2]:
!pip install --upgrade pip
!pip install --upgrade setuptools

런타임 다시 시작

In [3]:
!pip install -U auto-sklearn

# Auto-Sklearn
가장 유명한 파이썬 데이터 분석 패키지 중 하나인 Sklearn의 AutoML 버전이다.

AutoSklearn 2.0의 Classifier을 통해 이미지를 분류하는 모델을 자동으로 탐색하고(NAS), 하이퍼파라메터를 최적화한다(HPO).

In [4]:
import sklearn.datasets
import sklearn.metrics

from autosklearn.experimental.askl2 import AutoSklearn2Classifier

import tensorflow as tf
import numpy as np

## Data Loading
Kaggle의 Leaf Rice Disease Dataset을 이용한다.

모든 노트에서 데이터 처리하는 것이 번거로우므로, ndarray화 시킨 이미지 데이터셋을 구글 드라이브에 업로드한 후, 각 노트에서 불러와 사용했다.

dataset = imgset(small_rgb).npy.npz

* `dataset['x'].shape : (240, 360, 480, 3)` -> Numpy ndarray image data
* `dataset['y'].shape : (240, )` -> labels composed of 0, 1, 2

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dataset = np.load('./imgset(small_rgb).npy.npz')
data = dataset['x']
labels = dataset['y']

In [6]:
d1, d2, d3, d4 = data.shape
data_reshaped = data.reshape(d1, d2 * d3 * d4)

In [7]:
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(data_reshaped, labels, test_size=0.2)

## Build a classifier and Fit
AutoSklearn이 classification에 적합한 모델을 탐색한다.

코드가 간단한 것이 AutoML의 큰 장점인데, 모델 설계와 학습을 단 두 줄에 끝냈다.

시간과 자원을 고려해 최대 탐색 시간은 30분으로 제한했다.



무료 가용 ram을 13~16GB 지원하는 Google Colab에 비해 Paperspace는 무료 가용 ram을 무려 30GB나 지원한다.

따라서 메모리 제한을 넉넉하게 15GB로 주었다.

In [9]:
automl = AutoSklearn2Classifier(
    time_left_for_this_task=1800,
    memory_limit=15360,
    n_jobs=-1,
)
automl.fit(X_train, y_train, dataset_name='leaf_disease')

Using Backup selector


AutoSklearn2Classifier(memory_limit=15360, n_jobs=-1, per_run_time_limit=360,
                       time_left_for_this_task=1800)

## Print the pipelines constructed by auto-sklearn
show_models() 메소드를 통해 fit()을 진행하면서 AutoSklearn이 찾아낸 모델들을 볼 수 있다.

파이프라인 내부를 보면 모델 구조나 데이터 전처리, 경사하강 기법, 학습 속도 등을 찾아낸 것을 알 수 있다.

환경이 바뀌어서 그런지, Colab에서 작업한 버전보다 더 복잡한 모델을 만들어냈다.

In [10]:
print(automl.show_models())

[(0.160000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'extra_trees', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'minmax', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:extra_trees:bootstrap': 'False', 'classifier:extra_trees:criterion': 'gini', 'classifier:extra_trees:max_depth': 'None', 'classifier:extra_trees:max_features': 0.9959781561414611, 'classifier:extra_trees:max_leaf_nodes': 'None', 'classifier:extra_trees:min_impurity_decrease': 0.0, 'classifier:extra_trees:min_samples_leaf': 1, 'classifier:extra_trees:min_samples_split': 8, 'classifier:extra_trees:min_weight_fraction_leaf': 0.0, 'data_preprocessing:categorical_transformer:c

## Get accuracy of the final ensemble model
앙상블 기법으로 완성된 최종 모델의 성능을 테스트한다.

30분동안 학습한 결과 test_accuracy 87.5%를 달성했다.

In [11]:
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score: 0.875
